# **Spots analysis for csv files from TrackMate**

This notebook measures:

*   peak count (ch2)
*   peak distanced
* full withd of hal maximum (fwhm) of peaks

Written by Joanna Pylvänäinen

joanna.pylvanainen@abo.fi

In [ ]:
# @title #Mount GDrive

from google.colab import drive
drive.mount('/content/drive')

!pip install fastdtw


In [ ]:
# @title #Load useful functions


def normalize_to_01(series):
    """
    Normalize a given time series (1D numpy array) between 0 and 1.

    Parameters:
        series (numpy.ndarray): The time series to be normalized.

    Returns:
        numpy.ndarray: The normalized time series.
    """
    Imin = np.min(series)
    Imax = np.max(series)

    if Imax == Imin:
        return np.zeros_like(series)

    return (series - Imin) / (Imax - Imin)

# Function to normalize by maximum amplitude
def normalize_by_max_amplitude(series):
    return series / np.max(np.abs(series))



---


# **Part 2: Analysing spot data**



---



In [ ]:
# @title # Load spots and tracks tables
import pandas as pd
import glob
import os
import numpy as np

# Paths
Data_folder_path = '/content/drive/MyDrive/Kurppa/2025_reanalysis/Drifty_removed/1FUCCI_Osimertinib_tracking_images_frame48-132_no_split'  # @param {type: "string"}
Results_Folder = "/content/drive/MyDrive/Kurppa/2025_reanalysis/Cleaned_notebooks/results_spots"  # @param {type: "string"}

# Function to extract metadata and process the DataFrame
def process_dataframe(filepath, well_info, fov_info, file_name_without_ext):
    metadata = {
        'Well': well_info,
        'FOV': fov_info,
        'File Name': file_name_without_ext
    }

    metadata['Condition'] = np.select(
        [well_info in ['C02', 'C03', 'C04', 'C05', 'C06'],
         well_info in ['C07', 'C08', 'C09', 'C10', 'C11'],
         well_info in ['F02', 'F03', 'F04', 'F05', 'F06'],
         well_info in ['F07', 'F08', 'F09', 'F10', 'F11']],
        ['Control pool', 'Control single cell', 'Mutation #34', 'Mutation #38'],
        default='Unknown'
    )

    well_order = {'C02': 1, 'C03': 2, 'C04': 3, 'C05': 4, 'C06': 5,
                  'C07': 1, 'C08': 2, 'C09': 3, 'C10': 4, 'C11': 5,
                  'F02': 1, 'F03': 2, 'F04': 3, 'F05': 4, 'F06': 5,
                  'F07': 1, 'F08': 2, 'F09': 3, 'F10': 4, 'F11': 5}

    metadata['Repeat'] = f'{well_order.get(well_info, 1)}'

    return metadata

# Process spots data
spots_df_list = []
for filepath in glob.glob(Data_folder_path + '/*spots*.csv'):
    filename = os.path.basename(filepath)
    well_info = filename[0:3]
    fov_info = filename.split('_')[1]
    file_name_without_ext = os.path.splitext(filename)[0].replace('_tracking-spots', '')

    df = pd.read_csv(filepath, skiprows=[1, 2, 3])

    metadata = process_dataframe(filepath, well_info, fov_info, file_name_without_ext)
    for key, value in metadata.items():
        df[key] = value

    df['Unique_ID'] = file_name_without_ext + '_' + df['TRACK_ID'].astype(str)

    if 'TOTAL_INTENSITY_CH1' in df.columns and 'TOTAL_INTENSITY_CH2' in df.columns:
        df['Yellow'] = df['TOTAL_INTENSITY_CH1'] + df['TOTAL_INTENSITY_CH2']

    spots_df_list.append(df)

# Process tracks data
tracks_df_list = []
for filepath in glob.glob(Data_folder_path + '/*tracks*.csv'):
    filename = os.path.basename(filepath)
    well_info = filename[0:3]
    fov_info = filename.split('_')[1]
    file_name_without_ext = os.path.splitext(filename)[0].replace('_tracking-tracks', '')

    df = pd.read_csv(filepath, skiprows=[1, 2, 3])

    metadata = process_dataframe(filepath, well_info, fov_info, file_name_without_ext)
    for key, value in metadata.items():
        df[key] = value

    df['Unique_ID'] = file_name_without_ext + '_' + df['TRACK_ID'].astype(str)

    tracks_df_list.append(df)

# Merge all data
merged_spots_df = pd.concat(spots_df_list, ignore_index=True)
merged_tracks_df = pd.concat(tracks_df_list, ignore_index=True)

# Select relevant columns from merged_tracks_df for merging
tracks_info = merged_tracks_df[['Unique_ID', 'TRACK_DURATION', 'NUMBER_SPOTS']]

# Merge TRACK_DURATION and NUMBER_SPOTS into merged_spots_df based on Unique_ID
merged_spots_df = merged_spots_df.merge(tracks_info, on='Unique_ID', how='left')

# Save the final CSVs
merged_spots_df.to_csv(Results_Folder + '/' + 'merged_Spots.csv', index=False)
merged_tracks_df.to_csv(Results_Folder + '/' + 'merged_Tracks.csv', index=False)

print("Spots and Tracks files merged and saved successfully.")


# **Part 2.1: Peak Analysis**

In [ ]:
paramater_to_plot = 'TOTAL_INTENSITY_CH2'

#TOTAL_INTENSITY_CH1 TOTAL_INTENSITY_CH2 Yellow

# @title #Prepare and normalise the data (Fluo over time) of balanced dataset (example well F04_02)


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# Your normalize_to_01 function
def normalize_to_01(series):
    return (series - series.min()) / (series.max() - series.min())


# Filter data for the specific well "F02" and FOV "02"
filtered_df = merged_spots_df[(merged_spots_df['Well'] == 'F04') & (merged_spots_df['FOV'] == '02')]
filtered_df = filtered_df.groupby('TRACK_ID').filter(lambda x: len(x) >= 50)

# Group by 'TRACK_ID'
grouped = filtered_df.groupby('TRACK_ID')
valid_track_ids = [name for name, group in grouped if group[paramater_to_plot].max() > 10000]

# Filter DataFrame for valid TRACK_IDs
filtered_df = filtered_df[filtered_df['TRACK_ID'].isin(valid_track_ids)]

# Sort by 'TRACK_ID' and 'POSITION_T'
filtered_df.sort_values(['TRACK_ID', 'POSITION_T'], inplace=True)

# Rolling average
window_size = 3
filtered_df['Rolling_Avg'] = filtered_df.groupby('TRACK_ID')[paramater_to_plot].transform(
    lambda x: x.rolling(window=window_size).mean()
)

# Remove NaNs
filtered_df.dropna(subset=['Rolling_Avg'], inplace=True)

# Normalization
for track_id in filtered_df['TRACK_ID'].unique():
    track_data = filtered_df[filtered_df['TRACK_ID'] == track_id]
    filtered_df.loc[filtered_df['TRACK_ID'] == track_id, 'Rolling_Avg_normalised'] = normalize_to_01(track_data['Rolling_Avg'])

# Get 10 unique track IDs for plotting
selected_track_ids = filtered_df['TRACK_ID'].unique()[:10]

# Define the new folder name
new_folder = os.path.join(Results_Folder, 'normalized_track_data_plots')

# Create the new folder if it doesn't exist
os.makedirs(new_folder, exist_ok=True)

# Define the new folder name
new_folder = os.path.join(Results_Folder, 'normalized_track_data_plots')
os.makedirs(new_folder, exist_ok=True)
pdf_pages = PdfPages(os.path.join(new_folder, 'Track_Data_Plots_example well F04_02.pdf'))

# Create plots for the selected tracks
for track_id in selected_track_ids:
    track_data = filtered_df[filtered_df['TRACK_ID'] == track_id]

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # Plot raw data
    sns.lineplot(x='POSITION_T', y=paramater_to_plot, data=track_data, ax=axs[0])
    axs[0].set_title(f'Track {track_id} - Raw Data')

    # Plot rolling average
    sns.lineplot(x='POSITION_T', y='Rolling_Avg', data=track_data, ax=axs[1])
    axs[1].set_title(f'Track {track_id} - Rolling Avg')

    # Plot normalized rolling average
    sns.lineplot(x='POSITION_T', y='Rolling_Avg_normalised', data=track_data, ax=axs[2])
    axs[2].set_title(f'Track {track_id} - Normalized')

    plt.tight_layout()
    pdf_pages.savefig(fig)
    plt.show()
    plt.close(fig)

pdf_pages.close()


In [ ]:
# @title #Align peaks

import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import find_peaks


def evaluate_track(track_data):
    mean_intensity = np.mean(track_data)
    std_intensity = np.std(track_data)
    snr = mean_intensity / std_intensity if std_intensity != 0 else 0
    # Peak detection
    peaks, _ = find_peaks(track_data, height=0.5)  # Adjust height parameter as needed
    # Regularity (standard deviation of the distance between peaks)
    peak_distances = np.diff(peaks)
    regularity = np.std(peak_distances) if len(peak_distances) > 0 else 0
    return snr, len(peaks), regularity

# Get unique track IDs
unique_tracks = filtered_df['TRACK_ID'].unique()

# Evaluate all tracks
track_metrics = {}
for track_id in unique_tracks:
    track_data = filtered_df[filtered_df['TRACK_ID'] == track_id]['Rolling_Avg_normalised']
    snr, num_peaks, regularity = evaluate_track(track_data)
    track_metrics[track_id] = {'SNR': snr, 'NumPeaks': num_peaks, 'Regularity': regularity}

# Convert dict to DataFrame for easier manipulation
track_metrics_df = pd.DataFrame.from_dict(track_metrics, orient='index')

# Find the "best" track based on the metrics
best_tracks_df = track_metrics_df.loc[
    (track_metrics_df['NumPeaks'] == track_metrics_df['NumPeaks'].max())]

# Check if the DataFrame is empty
if not best_tracks_df.empty:
    best_track_id = best_tracks_df.index[0]
    print("Best Track ID:", best_track_id)
else:
    print("No track meets all the criteria.")

# Find the longest series
lengths = [len(filtered_df[filtered_df['TRACK_ID'] == track]) for track in unique_tracks]
longest_track = unique_tracks[np.argmax(lengths)]

# Initialize a list to store aligned time series
aligned_series_list = []

# Choose a reference track, you can change this
reference_track = longest_track
reference_df = filtered_df[filtered_df['TRACK_ID'] == best_track_id]
reference_series = reference_df[['POSITION_T', 'Rolling_Avg_normalised']].values

# Align all other tracks to the reference track
for track in unique_tracks[1:]:
    current_df = filtered_df[filtered_df['TRACK_ID'] == track]
    current_series = current_df[['POSITION_T', 'Rolling_Avg_normalised']].values

    # Perform DTW
    _, path = fastdtw(reference_series, current_series, dist=euclidean)

    # Extract the aligned indices
    idx_ref, idx_current = zip(*path)

    # Interpolate to match the reference series length
    interp_func = interp1d(current_series[list(idx_current), 0], current_series[list(idx_current), 1], kind='linear', fill_value="extrapolate")
    aligned_series = interp_func(reference_series[:, 0])


    # Store aligned time series
    aligned_series_list.append(aligned_series)

# Convert the list of aligned series into a 2D array
aligned_series_array = np.array(aligned_series_list)

# Plot the reference series and a few aligned series
plt.figure(figsize=(10, 6))
plt.plot(reference_series[:, 1], label=f'Reference {reference_track}')

# Plot first 5 aligned series
for i, aligned_series in enumerate(aligned_series_array[:15]):
    plt.plot(aligned_series, label=f'Aligned {unique_tracks[i + 1]}')  # i+1 because the reference track is not in aligned_series_array

plt.legend()
plt.title(paramater_to_plot)
plt.xlabel('Aligned Time')
plt.ylabel(paramater_to_plot)
plt.show()


In [ ]:
# @title #Average aligned peaks

import numpy as np
import matplotlib.pyplot as plt

# Convert the list of aligned series into a 2D array
aligned_series_array = np.array(aligned_series_list)

# Find the minimum length among all series in the array
min_length = min([len(series) for series in aligned_series_array])

# Crop each time series to this minimum length
cropped_series_array = np.array([series[:min_length] for series in aligned_series_array])

# Calculate mean and standard deviation along the rows
mean_series = np.mean(cropped_series_array, axis=0)
std_series = np.std(cropped_series_array, axis=0)

# Plot mean and standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_series, label='Mean', color='blue')
plt.fill_between(range(len(mean_series)), mean_series-std_series, mean_series+std_series,
                 color='blue', alpha=0.2, label='Standard Deviation')
plt.title('Mean and Standard Deviation of Aligned and Normalized Time Series')
plt.xlabel('Aligned Time')
plt.ylabel(paramater_to_plot)
plt.legend()
plt.show()


In [ ]:
# @title #Find the distance between peaks for selected condition

import pandas as pd
from scipy.signal import find_peaks
import numpy as np

# Initialize an empty dictionary to store peak distances for each TRACK_ID
peak_distances_dict = {}

# Loop through each unique TRACK_ID
for track_id in filtered_df['TRACK_ID'].unique():

    # Filter the DataFrame to get data for this specific TRACK_ID
    track_df = filtered_df[filtered_df['TRACK_ID'] == track_id]

    # Extract the Rolling_Avg and POSITION_T values
    rolling_avg_values = track_df['Rolling_Avg_normalised'].values
    position_t_values = track_df['POSITION_T'].values

    # Find peaks in the Rolling_Avg data
    peaks, _ = find_peaks(rolling_avg_values, height=0.8, distance=10)

    if len(peaks) < 2:  # Skip tracks with fewer than two peaks
        continue

    # Calculate the distances (in terms of POSITION_T) between consecutive peaks
    peak_positions = position_t_values[peaks]
    peak_distances = np.diff(peak_positions)

    # Store these distances in the dictionary
    peak_distances_dict[track_id] = peak_distances

# Convert the dictionary to a DataFrame for easier manipulation
peak_distances_df = pd.DataFrame.from_dict(peak_distances_dict, orient='index')
peak_distances_df.columns = [f'Distance_{i+1}' for i in range(peak_distances_df.shape[1])]

# Calculate the average peak distance for each track and store it in a new DataFrame
average_peak_distances = peak_distances_df.mean(axis=1).reset_index()
average_peak_distances.columns = ['TRACK_ID', 'Average_Peak_Distance']

print(average_peak_distances)


In [ ]:
# @title #Measure Peak duration

threshold = 0.5  # Set your threshold value here


def find_FWTM(x, y, threshold):
    max_y = max(y)
    threshold_y = max_y * threshold
    left_idx = np.where(y >= threshold_y)[0][0]
    right_idx = np.where(y >= threshold_y)[0][-1]
    FWTM = x[right_idx] - x[left_idx]
    return FWTM, x[left_idx], x[right_idx]

fwhm_dict = {}

# Loop through each unique TRACK_ID
num_plots = 0  # Number of plots generated

for track_id in filtered_df['TRACK_ID'].unique():  # Process all unique TRACK_IDs

    # Filter the DataFrame to get data for this specific TRACK_ID
    track_df = filtered_df[filtered_df['TRACK_ID'] == track_id]

    if num_plots >= 10:
        break  # Stop after plotting 10 tracks

    # Extract the Rolling_Avg and POSITION_T values
    rolling_avg_values = track_df['Rolling_Avg_normalised'].values
    position_t_values = track_df['POSITION_T'].values

    # Find peaks in the Rolling_Avg data
    peaks, _ = find_peaks(rolling_avg_values, height=0.5, distance=10)

    fwhm_list = []

    plt.figure()
    plt.plot(position_t_values, rolling_avg_values, label="Signal")

    for peak in peaks:

        start_idx = np.where(rolling_avg_values[:peak] < threshold)[0][-1] if np.any(rolling_avg_values[:peak] < threshold) else 0
        end_idx = np.where(rolling_avg_values[peak:] < threshold)[0][0] + peak if np.any(rolling_avg_values[peak:] < threshold) else len(rolling_avg_values) - 1

        # Update local_x and local_y based on threshold
        local_x = position_t_values[start_idx:end_idx]
        local_y = rolling_avg_values[start_idx:end_idx]

        FWHM, left, right = find_FWTM(local_x, local_y, 0.4)  # Here, the threshold is 0.5, i.e., half maximum

        fwhm_list.append(FWHM)

        plt.scatter(position_t_values[peak], rolling_avg_values[peak], color='red')  # Mark the peak
        plt.axvline(x=left, color='green', linestyle='--')  # Mark the left half-maximum
        plt.axvline(x=right, color='green', linestyle='--')  # Mark the right half-maximum

    plt.title(f'Track {track_id}')
    plt.xlabel('POSITION_T')
    plt.ylabel('Rolling Avg Intensity')
    plt.legend()
    plt.show()

    # Save the list of FWHM values for this track to the dictionary
    fwhm_dict[track_id] = fwhm_list

    num_plots += 1  # Increment the number of plots generated

# Convert the dictionary to a DataFrame for easier manipulation
fwhm_df = pd.DataFrame.from_dict(fwhm_dict, orient='index')
fwhm_df.columns = [f'FWHM_{i+1}' for i in range(fwhm_df.shape[1])]

# **Part 2.2: Analyse all conditions**

In [ ]:
# @title #Measure distance between two G1 peaks for all conditions (this can take up to 1h depending on the size of data)

import warnings
warnings.filterwarnings('ignore')

# Move these to the top of the script as configuration parameters
PEAK_DETECTION_CONFIG = {
    'height': 10000,  # Peak_height
    'distance': 5,    # Peak_distance
    'rolling_window': 3,  # window_size
    'signal_background': 5000,  # Signal_background
    'fwtm_threshold': 0.4  # Threshold for FWTM calculation
}

paramater_to_plot = 'TOTAL_INTENSITY_CH2'
window_size = 3  # for rolling average
Signal_background = 5000  # for filtering
Peak_height = 10000  # for peak detection
Peak_distance = 5

import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import seaborn as sns

threshold = 5000  # Set your threshold value here

fwhm_dict = {}
peak_distances_dict = {}

def find_FWTM(x, y, threshold):
    max_y = max(y)
    threshold_y = max_y * threshold
    left_idx = np.where(y >= threshold_y)[0][0]
    right_idx = np.where(y >= threshold_y)[0][-1]
    FWTM = x[right_idx] - x[left_idx]
    return FWTM, x[left_idx], x[right_idx]


def normalize_to_01(series):
    return (series - series.min()) / (series.max() - series.min())

# Get unique wells, FOVs, Conditions, and Repeats
unique_wells = merged_spots_df['Well'].unique()
unique_fovs = merged_spots_df['FOV'].unique()
unique_conditions = merged_spots_df['Condition'].unique()
unique_repeats = merged_spots_df['Repeat'].unique()

# Initialize an empty DataFrame to store the overall results
final_results_fwhm = pd.DataFrame()
final_results_average_peak_distances = pd.DataFrame()
peak_info_df = pd.DataFrame(columns=['Unique_ID', 'Well', 'FOV', 'Has_Peak'])

# Initialize a dictionary to store peak counts
peak_counts_dict = {}

# Loop through each well and each FOV
for well in unique_wells:
    for fov in unique_fovs:
        print(well + fov + " is being processed")

        # Create the peak_plots folder if it doesn't exist
        peak_plots_folder = os.path.join(Results_Folder, "peak_plots")
        os.makedirs(peak_plots_folder, exist_ok=True)

        # Include the peak_plots folder in the PDF file path
        pdf_file_path = os.path.join(
            peak_plots_folder, f"{well}_{fov}_All_Tracks_Peaks.pdf")

        # Filter data for the specific well and FOV
        filtered_df = merged_spots_df[(merged_spots_df['Well'] == well) & (
            merged_spots_df['FOV'] == fov)]
        filtered_df = filtered_df.groupby('TRACK_ID').filter(
            lambda x: len(x) >= 50)

        # Group by 'TRACK_ID'
        grouped = filtered_df.groupby('TRACK_ID')
        valid_track_ids = [name for name, group in grouped if group[paramater_to_plot].max() > 10000]

        # Filter DataFrame for valid TRACK_IDs
        filtered_df = filtered_df[filtered_df['TRACK_ID'].isin(valid_track_ids)]

        with PdfPages(pdf_file_path) as pdf_pages:  # The with statement

            # Sort by 'TRACK_ID' and 'POSITION_T'
            filtered_df.sort_values(['TRACK_ID', 'POSITION_T'], inplace=True)

            # Apply the rolling mean within each 'TRACK_ID' group
            filtered_df['Rolling_Avg'] = filtered_df.groupby('TRACK_ID')[paramater_to_plot].transform(
                lambda x: x.rolling(window=window_size).mean())

            # Optional: Remove rows with NaN (these are the rows where the rolling mean could not be computed)
            filtered_df.dropna(subset=['Rolling_Avg'], inplace=True)

            # Normalization
            for track_id in filtered_df['TRACK_ID'].unique():
                track_data = filtered_df[filtered_df['TRACK_ID']
                                          == track_id]
                filtered_df.loc[filtered_df['TRACK_ID'] == track_id, 'Rolling_Avg_normalised'] = normalize_to_01(
                    track_data['Rolling_Avg'])

        # Get unique TRACK_IDs
        print(well + fov +
              " Data preprocessed and normalised, looking for peaks")
        unique_track_ids = filtered_df['TRACK_ID'].unique()

        # Sort by 'TRACK_ID' and 'POSITION_T' if not already sorted
        filtered_df.sort_values(['TRACK_ID', 'POSITION_T'], inplace=True)

        # Loop through each unique TRACK_ID
        for track_id in filtered_df['Unique_ID'].unique():
            # Filter the DataFrame to get data for this specific TRACK_ID
            track_df = filtered_df[filtered_df['Unique_ID'] == track_id]

            # Get the repeat for the current track_id
            repeat = track_df['Repeat'].iloc[0]

            # Extract the Rolling_Avg and POSITION_T values
            rolling_avg_values = track_df['Rolling_Avg'].values
            position_t_values = track_df['POSITION_T'].values
            # Find peaks in the Rolling_Avg data
            peaks, _ = find_peaks(
                rolling_avg_values, height=Peak_height, distance=Peak_distance)

            # Count the number of peaks
            num_peaks = len(peaks)
            peak_counts_dict[track_id] = num_peaks

            has_peak = len(peaks) > 0

            # Append this information to peak_info_df
            peak_info_df = pd.concat([peak_info_df, pd.DataFrame([{
                'Unique_ID': track_id,
                'Well': well,
                'FOV': fov,
                'Has_Peak': has_peak,
                'Repeat': repeat # Add Repeat here
            }])], ignore_index=True)

            fwhm_list = []
            fig, ax = plt.subplots()
            ax.plot(position_t_values, rolling_avg_values, label="Signal")

            for peak in peaks:
                # Find the neighborhood around the peak
                start_idx = peak
                end_idx = peak

                # Find the left boundary where the signal drops below the threshold
                while start_idx > 0 and rolling_avg_values[start_idx] >= threshold:
                    start_idx -= 1

                # Find the right boundary where the signal drops below the threshold
                while end_idx < len(rolling_avg_values) - 1 and rolling_avg_values[end_idx] >= threshold:
                    end_idx += 1

                # Extract local_x and local_y based on these boundaries
                local_x = position_t_values[start_idx:end_idx]
                local_y = rolling_avg_values[start_idx:end_idx]

                FWHM, left, right = find_FWTM(
                    local_x, local_y, 0.4)  # Here, the threshold is 0.5, i.e., half maximum
                fwhm_list.append(FWHM)

                plt.scatter(position_t_values[peak],
                            rolling_avg_values[peak], color='red')  # Mark the peak
                plt.axvline(x=left, color='green',
                            linestyle='--')  # Mark the left half-maximum
                plt.axvline(x=right, color='green',
                            linestyle='--')  # Mark the right half-maximum
                plt.title(f'Track {track_id}')
                plt.xlabel('POSITION_T')
                plt.ylabel('Rolling Avg Intensity')
                plt.legend()
                pdf_pages.savefig(fig)

            # Just collect the data in dictionaries
            if fwhm_list:  # if there are FWHM values
                fwhm_dict[track_id] = {
                    'fwhm_list': fwhm_list,
                    'Well': well,
                    'FOV': fov,
                    'Repeat': repeat,
                    'Condition': merged_spots_df[merged_spots_df['Well'] == well]['Condition'].iloc[0]
                }

            if len(peaks) >= 2:
                peak_distances = np.diff(position_t_values[peaks])
                peak_distances_dict[track_id] = {
                    'distances': peak_distances,
                    'Well': well,
                    'FOV': fov,
                    'Repeat': repeat,
                    'Condition': merged_spots_df[merged_spots_df['Well'] == well]['Condition'].iloc[0]
                }

# Add 'Condition' column to peak_info_df
peak_info_df = pd.merge(
peak_info_df, merged_spots_df[['Unique_ID', 'Condition']], on='Unique_ID', how='left') #This is now redundant

# After all loops are done, create the final DataFrames once
final_results_fwhm = pd.DataFrame([
    {
        'Unique_ID': track_id,
        'Average_fwhm': np.mean(data['fwhm_list']),
        'Well': data['Well'],
        'FOV': data['FOV'],
        'Repeat': data['Repeat'],
        'Condition': data['Condition']
    }
    for track_id, data in fwhm_dict.items()
])

final_results_average_peak_distances = pd.DataFrame([
    {
        'Unique_ID': track_id,
        'Average_Peak_Distance': np.mean(data['distances']),
        'Well': data['Well'],
        'FOV': data['FOV'],
        'Repeat': data['Repeat'],
        'Condition': data['Condition']
    }
    for track_id, data in peak_distances_dict.items()
])

final_results_fwhm.to_csv(f"{Results_Folder}/fwhm_info_all.csv", index=False)
peak_info_df.to_csv(f"{Results_Folder}/peak_info_all.csv", index=False)

# Create a DataFrame from the peak counts dictionary
peak_counts_df = pd.DataFrame.from_dict(peak_counts_dict, orient='index', columns=['Num_Peaks'])
peak_counts_df.index.name = 'Unique_ID'

# Check if 'Num_Peaks' column already exists in merged_spots_df
if 'Num_Peaks' in merged_spots_df.columns:
    # If it exists, drop the existing 'Num_Peaks' column before merging
    merged_spots_df = merged_spots_df.drop(columns=['Num_Peaks'])

# Merge with peak_counts_df
merged_spots_df = pd.merge(merged_spots_df, peak_counts_df, on='Unique_ID', how='left')

# Initialize the PDF
with PdfPages(f"{Results_Folder}/Combined_Plots.pdf") as pdf_pages:

    # Create a single figure with 2 subplots for boxplots
    fig, axes = plt.subplots(2, 1, figsize=(10, 10))

    # Prepare the metrics to plot
    metrics_to_plot = ['Average_fwhm', 'Average_Peak_Distance']
    data_frames_to_use = [final_results_fwhm, final_results_average_peak_distances]

    # Loop over both axes and metrics for boxplots
    for ax, metric, df in zip(axes, metrics_to_plot, data_frames_to_use):

        # Save this data to a CSV file
        df[['Unique_ID', 'Well', 'FOV', 'Condition','Repeat', metric]].to_csv( # Add Repeat here
            f"{Results_Folder}/data_for_{metric}.csv", index=False)

        # Create the boxplot using 'Condition' for grouping
        sns.boxplot(x='Condition', y=metric,
                    data=df, ax=ax, color='lightgray')

        # Set title and labels
        ax.set_title(f"{metric}")
        ax.set_xlabel('Condition')  # Changed to 'Condition'
        ax.set_ylabel(metric)

    # Save the figure to a PDF page
    plt.tight_layout()
    pdf_pages.savefig(fig)
    plt.close(fig)  # Close this figure before plotting the next one

    # Create a new figure for the barplot
    fig, ax = plt.subplots(figsize=(20, 5))

    # Assuming peak_info_df is your DataFrame
    peak_info_df['Has_Peak'] = peak_info_df['Has_Peak'].astype(int)
    agg_df = peak_info_df.groupby('Condition')[
        'Has_Peak'].mean().reset_index()  # Group by 'Condition'

    # Create the bar plot using 'Condition' for grouping
    sns.barplot(x='Condition', y='Has_Peak',
                data=agg_df, ax=ax, color='lightblue')

    # Set title and labels
    ax.set_title('Proportion of Tracks with Peaks by Condition')  # Changed to 'Condition'
    ax.set_xlabel('Condition')  # Changed to 'Condition'
    ax.set_ylabel('Proportion of Tracks with Peaks')

    # Save the boxplot figure
    plt.tight_layout()
    pdf_pages.savefig(fig)
    plt.close(fig)

# Save the updated merged_spots_df
merged_spots_df.to_csv(Results_Folder + '/' + 'merged_Spots_with_NumPeaks.csv', index=False)

In [ ]:
# @title #Plot proportion of Tracks with Peaks by Well (QC)


import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming peak_info_df is your DataFrame with columns 'Well', 'FOV', and 'Has_Peak'

# Convert boolean to int for easier plotting and calculation
peak_info_df['Has_Peak'] = peak_info_df['Has_Peak'].astype(int)

# Calculate the mean of Has_Peak per Well, which gives the proportion of True values
agg_df = peak_info_df.groupby('Well')['Has_Peak'].mean().reset_index()

# Create a new figure
plt.figure(figsize=(20, 5))

# Sort the 'Group' column alphabetically
group_order = merged_spots_df['Well'].sort_values().unique()

# Create the bar plot for showing the proportion of True values in each Well
sns.barplot(x='Well', y='Has_Peak', data=agg_df, color='lightblue', order=group_order)

# Set title and labels
plt.title('Proportion of Tracks with Peaks by Well')
plt.xlabel('Well')
plt.ylabel('Proportion of Tracks with Peaks')
plt.savefig(os.path.join(Results_Folder, 'QC/Proportion of Tracks with Peaks per condition.pdf'), bbox_inches='tight') # This line saves the heatmap

# Show the plot
plt.show()

---


# **Move to plotting notebook from here**



---